In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
from bs4 import BeautifulSoup
import pickle

In [5]:
sentiment = pickle.load(open('nlp.pkl','rb'))
vectorizer = pickle.load(open('transform.pkl','rb'))

In [6]:
df = pd.read_csv('TMDB_Final.csv')

In [56]:
df[df['title']=='Batman (1989)']

,Unnamed: 0.1,Unnamed: 0,id,title,overview,popularity,vote_average,vote_count,release_date,keywords,genres,cast,crew
2319,2319,2319,268,Batman (1989),Batman must face his most ruthless nemesis whe...,53.219,7.2,5959,1989-06-23,"['double life', 'dc comics', 'dual identity', ...","['Fantasy', 'Action']","['Michael Keaton', 'Jack Nicholson', 'Kim Basi...",['Tim Burton']


In [8]:
main_features = df[['id','title','genres','overview','crew','keywords','cast','vote_average']]

In [9]:
main_features_copy = main_features.copy()
main_features_copy.dropna(inplace=True)

In [10]:
main_features_copy['title'].duplicated().sum()

2

In [11]:
main_features_copy['genres']=main_features_copy['genres'].apply(lambda x : x.replace('Science Fiction','Sci-Fi'))

In [12]:
main_features_copy['genres']=main_features_copy['genres'].apply(lambda x:x.replace("'",'').replace(" ",'').strip('[]'))
main_features_copy['cast']=main_features_copy['cast'].apply(lambda x:x.replace("'",'').replace(" ",'').strip('[]'))
main_features_copy['keywords']=main_features_copy['keywords'].apply(lambda x:x.replace("'",'').replace(" ",'').strip('[]'))   

In [13]:
cast = main_features_copy['cast']

In [14]:
cast[0]

'ShahRukhKhan,Kajol,AmrishPuri,AnupamKher,SatishShah,FaridaJalal,AchalaSachdev'

In [15]:
cast[1]

'TimRobbins,MorganFreeman,BobGunton,WilliamSadler,ClancyBrown,GilBellows,MarkRolston'

In [16]:
main_features_copy

,id,title,genres,overview,crew,keywords,cast,vote_average
0,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second...",['Aditya Chopra'],,"ShahRukhKhan,Kajol,AmrishPuri,AnupamKher,Satis...",8.7
1,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...,['Frank Darabont'],"prison,corruption,policebrutality,prisoncell,d...","TimRobbins,MorganFreeman,BobGunton,WilliamSadl...",8.7
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o...",['Francis Ford Coppola'],"italy,lossoflovedone,loveatfirstsight,italiana...","MarlonBrando,AlPacino,JamesCaan,RobertDuvall,R...",8.7
3,724089,Gabriel's Inferno Part II,Romance,Professor Gabriel Emerson finally learns the t...,['Tosca Musk'],basedonnovelorbook,"MelanieZanetti,GiulioBerruti,JamesAndrewFraser...",8.6
4,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...,['Steven Spielberg'],"basedonnovelorbook,factory,concentrationcamp,h...","LiamNeeson,BenKingsley,RalphFiennes,CarolineGo...",8.6
...,...,...,...,...,...,...,...,...
9475,21435,French Fried Vacation 3,Comedy,"After the Club Med and skiing, what happened t...",['Patrice Leconte'],"holiday,sardinia,italy,sequel,reunion,luxuryhotel","JosianeBalasko,MichelBlanc,Marie-AnneChazel,Ch...",4.1
9476,17711,The Adventures of Rocky & Bullwinkle,"Family,Adventure,Animation,Comedy,Fantasy",Rocky and Bullwinkle have been living off the ...,['Des McAnuff'],"helicopter,police,cartoon,roadtrip,carcrash,ja...","ReneRusso,JasonAlexander,PiperPerabo,RandyQuai...",4.1
9477,17532,S. Darko,"Thriller,Crime,Sci-Fi,Mystery","S. Darko follows Samantha Darko, the younger s...",['Chris Fisher'],"sequel,stranded,endoftheworld,shatteredhome,bi...","DaveighChase,ElizabethBerkley,BrianaEvigan,Jam...",4.1
9478,13908,The Master of Disguise,"Comedy,Family",A sweet-natured Italian waiter named Pistachio...,['Perry Andelin Blake'],"disguise,aftercreditsstinger,duringcreditsstinger","DanaCarvey,BrentSpiner,JenniferEsposito,Harold...",4.0


In [17]:
main_features_copy['cast']=main_features_copy['cast'].apply(lambda x:x.strip('[]'))
main_features_copy['genres']=main_features_copy['genres'].apply(lambda x:x.strip('[]'))
main_features_copy['keywords']=main_features_copy['keywords'].apply(lambda x:x.strip('[]'))

In [18]:
main_features_copy

,id,title,genres,overview,crew,keywords,cast,vote_average
0,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second...",['Aditya Chopra'],,"ShahRukhKhan,Kajol,AmrishPuri,AnupamKher,Satis...",8.7
1,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...,['Frank Darabont'],"prison,corruption,policebrutality,prisoncell,d...","TimRobbins,MorganFreeman,BobGunton,WilliamSadl...",8.7
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o...",['Francis Ford Coppola'],"italy,lossoflovedone,loveatfirstsight,italiana...","MarlonBrando,AlPacino,JamesCaan,RobertDuvall,R...",8.7
3,724089,Gabriel's Inferno Part II,Romance,Professor Gabriel Emerson finally learns the t...,['Tosca Musk'],basedonnovelorbook,"MelanieZanetti,GiulioBerruti,JamesAndrewFraser...",8.6
4,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...,['Steven Spielberg'],"basedonnovelorbook,factory,concentrationcamp,h...","LiamNeeson,BenKingsley,RalphFiennes,CarolineGo...",8.6
...,...,...,...,...,...,...,...,...
9475,21435,French Fried Vacation 3,Comedy,"After the Club Med and skiing, what happened t...",['Patrice Leconte'],"holiday,sardinia,italy,sequel,reunion,luxuryhotel","JosianeBalasko,MichelBlanc,Marie-AnneChazel,Ch...",4.1
9476,17711,The Adventures of Rocky & Bullwinkle,"Family,Adventure,Animation,Comedy,Fantasy",Rocky and Bullwinkle have been living off the ...,['Des McAnuff'],"helicopter,police,cartoon,roadtrip,carcrash,ja...","ReneRusso,JasonAlexander,PiperPerabo,RandyQuai...",4.1
9477,17532,S. Darko,"Thriller,Crime,Sci-Fi,Mystery","S. Darko follows Samantha Darko, the younger s...",['Chris Fisher'],"sequel,stranded,endoftheworld,shatteredhome,bi...","DaveighChase,ElizabethBerkley,BrianaEvigan,Jam...",4.1
9478,13908,The Master of Disguise,"Comedy,Family",A sweet-natured Italian waiter named Pistachio...,['Perry Andelin Blake'],"disguise,aftercreditsstinger,duringcreditsstinger","DanaCarvey,BrentSpiner,JenniferEsposito,Harold...",4.0


In [19]:
main_features_copy['tags']=main_features_copy['overview'] + main_features_copy['genres'] + main_features_copy['keywords'] + main_features_copy['cast'] + main_features_copy['crew']

In [20]:
new_df = main_features_copy[['title','tags']]

In [21]:
new_df
new_df_copy = new_df.copy()

In [22]:
from nltk import PorterStemmer
ps = PorterStemmer()
def stem(x):
    if pd.notna(x):
        L = []
        for i in x.split():
            L.append(ps.stem(i))
        return " ".join(L)
    else:
        return x 

In [23]:
new_df_copy['tags'] = new_df_copy['tags'].apply(stem)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [25]:
new_df_copy['tags'].fillna('', inplace=True)  

In [26]:
vectors = cv.fit_transform(new_df_copy['tags']).toarray()


In [27]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
similarity = cosine_similarity(vectors)

In [29]:
sorted_sim = sorted(list(enumerate(similarity[0])),reverse=True,key = lambda x:x[1])[1:12]

In [33]:
API_KEY='75d6f4d358afd80023fc135735da8af5'


In [48]:
def get_movie_id(movie):
    base_url = 'https://api.themoviedb.org/3/search/movie'
    params = {
        'api_key': API_KEY,
        'query': movie
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if 'results' in data and data['results']:
        movie_id = data['results'][0]['id']
        return movie_id
    else:
        print(f"No results found for '{movie}'")


In [59]:
def get_movie_details(movie):
    genre = (df.loc[df['title']==movie]).to_string(header=False,index=False,columns=['genres']).strip('[]').replace("'",'')
    print(genre)
    overview = (df.loc[df['title']==movie]).to_string(header=False,index=False,columns=['overview']).replace(",","")
    overview  = overview.strip('[]')
    release_date = (df.loc[df['title']==movie]).to_string(header=False,index=False,columns=['release_date'])
    formatted_date = pd.to_datetime(release_date).strftime('%d %B %Y')
    ratings = (df.loc[df['title']==movie]).to_string(header=False,index=False,columns=['vote_average'])
    print(ratings)
    return genre,overview,formatted_date,ratings

In [60]:
get_movie_details('Batman (1989)')

Fantasy, Action
7.2


('Fantasy, Action',
 'Batman must face his most ruthless nemesis when a deformed madman calling himself "The Joker" seizes control of Gotham\'s criminal underworld.',
 '23 June 1989',
 '7.2')

In [82]:
def get_castcrew(movie_id):
    response = requests.get('http://api.themoviedb.org/3/movie/{}?api_key=75d6f4d358afd80023fc135735da8af5&append_to_response=credits'.format(movie_id))
    data = response.json()
    dets =  data['credits']['cast'][0:10]
    ids = [i['id'] for i in dets]
    details= []
    character = [i['character'] for i in dets]
    for i in dets:
        if type(i) is not None:
            poster = ["https://image.tmdb.org/t/p/original"+i['profile_path'] for i in dets]

        for i in dets:
            i['profile_path'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFXT94RTDPEqHrcC1pVOjBpVUMzQ4dZ3QgOfJWQp2aLSOxEgQGQmu_jsgXu1czdhE1tzo&usqp=CAU'
            poster = ["https://image.tmdb.org/t/p/original"+i['profile_path'] for i in dets]


    for i in ids:
        response2 = requests.get('https://api.themoviedb.org/3/person/{}?api_key=75d6f4d358afd80023fc135735da8af5'.format(i))
        data2 = response2.json()
        details.append(data2)
    name = [i['name'] for i in details]

    dets = {"name":name,"poster":poster,"character":character}
    return poster

In [97]:
API_KEY='75d6f4d358afd80023fc135735da8af5'


In [98]:
def get_reviews(movie_id):
    reviews = []
    details_url = f'https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={API_KEY}'
    details_response = requests.get(details_url)
    details_data = details_response.json()       
    num_reviews = min(6, len(details_data['results']))
    print(num_reviews) 
    for i in range(num_reviews):
        reviews.append(details_data['results'][i]['content'])
    if len(reviews)==0:
        return 'No reviews Found'
    
    rev = np.array(reviews)
    n = vectorizer.transform(rev)
    sent = sentiment.predict(n)
    new_sent = sent.tolist()
        
    for i in range(len(new_sent)):
        if new_sent[i] == 0:
            new_sent[i] = 'Negative'
        else:
            new_sent[i] = 'Positive'
                
    return reviews, new_sent
    

In [99]:
get_reviews(557)

6


(["Sam Raimi's Spider-Man captures the spirit of the comic book hero and it's still one of my all time favourite superhero movies. It lays out the perfect blueprint for a superhero origin story that I love revisiting every year or so. I've always been able to relate to Peter Parker growing up. Tobey Maguire and Co. delivered a great interpretation of the characters in a way that still can not be matched.",
  'So many Spiderman movies out there but this will always be the original. Though the other ones have better CGI and maybe action sequences, none can match the plot this one has.',
  "This is one of the few films that you can watch when you're bored and still be excited about what's going to happen (even though you've watched it over and over already and you've memorized almost all of the lines).",
  'I keep telling people that this is the real Spider-Man and I will always stand by that. Tobey Maguire the real MVP!',
  'Films from the 2000s really are way different from the ones we 

In [88]:
def fetch_poster(movie_id):
    response= requests.get('https://api.themoviedb.org/3/movie/{}?api_key=75d6f4d358afd80023fc135735da8af5&language=en-US'.format(movie_id))
    data = response.json()
    try:
        path = 'https://image.tmdb.org/t/p/w500'+data['poster_path']
        return path
    except:
        pth = 'https://img.pikbest.com/backgrounds/20200428/film-and-television-festival-retro-film-poster_2824252.jpg!f305cw'
        return pth

In [89]:
def get_trailer(movie_id):
    response = requests.get('http://api.themoviedb.org/3/movie/{}?api_key=75d6f4d358afd80023fc135735da8af5&append_to_response=videos'.format(movie_id))
    data = response.json()
    try:
        key = data['videos']['results']
        return key
    except:
        data['videos']['results'] = 'https://images.wondershare.com/recoverit/article/2020/03/Video_unavailable_Img_1.jpg'
        return data['videos']['results']


In [100]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x:x[1])[1:12]
    titles= []
    ip_id = (df.loc[df['title']==movie]).to_string(header=False,index=False,columns=['id'])
    posters =[fetch_poster(ip_id)]
    trailer  = get_trailer(ip_id)
    details = get_movie_details(movie)
    creds = get_castcrew(ip_id)
    reviews = get_reviews(ip_id)
    for i in movie_list:
        title = new_df.iloc[i[0]].title
        movie_id = df.iloc[i[0]].id
        titles.append(title)
        posters.append(fetch_poster(movie_id))
        

    return titles



In [103]:
recommend('Batman (1966)')

0


['The Taking of Pelham One Two Three',
 "Mother's Day (2010)",
 'I Spit on Your Grave',
 'Dead Man Down',
 'Gotti',
 'Blood Ties',
 'Joker',
 'Hobo with a Shotgun',
 'Texas Killing Fields',
 'Funny Games (2007)',
 'Tracers']

In [102]:
import re

In [ ]:
def cast_poster(cast_id):
     try:
        path = 'https://image.tmdb.org/t/p/w500'+data['poster_path']
        return path
    except:
        pth = 'https://img.pikbest.com/backgrounds/20200428/film-and-television-festival-retro-film-poster_2824252.jpg!f305cw'
        return pth

In [ ]:
def cast_details(cast):
    
    response = requests.get('https://api.themoviedb.org/3/search/person?api_key=75d6f4d358afd80023fc135735da8af5&query={}'.format(cast))
    data = response.json()
    cast_id = data['results'][0]['id']
    response2 = requests.get('https://api.themoviedb.org/3/person/{}?api_key=75d6f4d358afd80023fc135735da8af5'.format(cast_id))
    data2 = response2.json()
    name = data2['name']
    bio = data2['biography']
    pob = data2['place_of_birth'] 
    try:
    poster ='https://image.tmdb.org/t/p/original'+data2['profile_path']
    
    dob = data2['birthday']    
    
    movies_with_actor = main_features[main_features['cast'].apply(lambda x: cast in x)]
    movies=movies_with_actor['title']
    movie_names = [re.sub(r'^\d+\s+', '', movie) for movie in movies]
    posters=[]
    for movie in movie_names:
        ip_id = (new_df.loc[new_df['title']==movie]).to_string(header=False,index=False,columns=['id'])
        posters.append(fetch_poster(ip_id))
   
    return posters

In [ ]:
cast_details('Paul Dano')

In [ ]:
def get_runtime(movie_id):
    details_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key=75d6f4d358afd80023fc135735da8af5'
    details_response = requests.get(details_url)
    details_data = details_response.json()
    
    if 'runtime' in details_data:
        runtime = details_data['runtime']
    
    return runtime

In [ ]:
ids = df['id']
posters = []
for i in ids:
    posters.append(fetch_poster(i))
    
posters

In [ ]:
posters[1000]

In [ ]:
ip_id = (new_df.loc[new_df['title']=='Hidden Kisses']).to_string(header=False,index=True,columns=['id'])
ip_id


In [ ]:
fetch_poster(1000)

In [ ]:
print(posters)

In [ ]:
new_df

In [ ]:
t = pd.read_csv('TMDB_revised.csv')

In [ ]:
ndf= t[['id','title','genres','runtime']]

In [ ]:
ndf[['Changeling']

In [ ]:
ndf['posters'] = posters

In [ ]:
ndf

In [ ]:
ndf.to_csv('Filter.csv')